In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import (
    VGG16, VGG19, ResNet152V2, MobileNetV2, InceptionV3, Xception,
    DenseNet169, EfficientNetB0, EfficientNetB2, EfficientNetB3, EfficientNetB5, EfficientNetB7
)
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# Hyperparameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001
NUM_CLASSES = 16

In [ ]:
train_dir = '/content/drive/MyDrive/Research/Data/Dataset/train'
val_dir = '/content/drive/MyDrive/Research/Data/Dataset/validate'
test_dir = '/content/drive/MyDrive/Research/Data/Dataset/test'

In [ ]:
# Data Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


In [ ]:
# function to create models
def create_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(NUM_CLASSES, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
# List of models
models = [
    (VGG16, 'VGG16'),
    (VGG19, 'VGG19'),
    (ResNet152V2, 'ResNet152V2'),
    (MobileNetV2, 'MobileNetV2'),
    (InceptionV3, 'InceptionV3'),
    (Xception, 'Xception'),
    (DenseNet169, 'DenseNet169'),
    (EfficientNetB0, 'EfficientNetB0'),
    (EfficientNetB2, 'EfficientNetB2'),
    (EfficientNetB3, 'EfficientNetB3'),
    (EfficientNetB5, 'EfficientNetB5'),
    (EfficientNetB7, 'EfficientNetB7')
]


In [ ]:
# Training and Evaluating the models
for model_class, model_name in models:
    print(f"Training {model_name}...")

    base_model = model_class(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
    model = create_model(base_model)

    # Train the model
    model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=val_generator,
        validation_steps=val_generator.samples // BATCH_SIZE
    )


In [ ]:
# Evaluate the model on test data
    loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
    print(f"{model_name} Test Accuracy: {accuracy:.4f}")